In [ ]:
class Point:
  def __init__(self, x, y):
    self.x = x
    self.y = y

def printPointList(points):
  print()
  for p in points:
    printPoint(p)

def printPoint(point):
  print("(", point.x, ", ", point.y, ")", sep="", end=" ")

def receiveInput():
    inputPoints = input().split()

    points = []

    for i in range(0, len(inputPoints) - 1, 2):
      points.append(Point(int(inputPoints[i]), int(inputPoints[i + 1])))

# points = receiveInput()

points = [(0, 3), (1, 2), (4, 4), (0, 0), (3, 1), (3, 3), (4, 2), (-2, 1)]

points = [Point(x, y) for x, y in points]

In [ ]:
def findBottomMostPoint(points):
  bottomMostPoint = points[0]

  for p in points:
    if p.y < bottomMostPoint.y or (p.y == bottomMostPoint.y and p.x < bottomMostPoint.x):
      bottomMostPoint = p

  return bottomMostPoint

bottomMostPoint = findBottomMostPoint(points) 
printPoint(bottomMostPoint)

printPointList(points)
points.remove(bottomMostPoint)
printPointList(points)

In [ ]:
from functools import cmp_to_key

def findOrientation(p0, p1, p2):
  crossProduct = (p1.x - p0.x) * (p2.y - p0.y) - (p2.x - p0.x) * (p1.y - p0.y)
  if crossProduct == 0:
    return "collinear"
  elif crossProduct > 0:
    return "right"
  else:
    return "left"

def findDistance(p0, p1):
  return ((p1.x - p0.x) ** 2 + (p1.y - p0.y) ** 2) ** 0.5

def compare(p1, p2):
  orientation = findOrientation(bottomMostPoint, p1, p2)
  if orientation == "collinear":
    if findDistance(bottomMostPoint, p1) <= findDistance(bottomMostPoint, p2):
      return -1
    else:
      return 1
  elif orientation == "right":
    return -1
  else:
    return 1

sortedPoints = sorted(points, key = cmp_to_key(compare))

def removeCollinearPoints(sortedPoints):
  i = 0
  while i < len(sortedPoints) - 1:
    if findOrientation(bottomMostPoint, sortedPoints[i], sortedPoints[i + 1]) == "collinear":
      sortedPoints.pop(i)
    else:
      i += 1

removeCollinearPoints(sortedPoints)
printPointList(sortedPoints)

sortedPoints = [bottomMostPoint] + sortedPoints
printPointList(sortedPoints)


In [ ]:
def findConvexHull(sortedPoints):
    if len(sortedPoints) < 3:
      print("Convex hull is not possible!")

    convexHull = [sortedPoints[0], sortedPoints[1], sortedPoints[2]]
    for i in range(3, len(sortedPoints)):
        while findOrientation(convexHull[-2], convexHull[-1], sortedPoints[i]) != "right":
            convexHull.pop()
        convexHull.append(sortedPoints[i])
    return convexHull

convexHull = findConvexHull(sortedPoints)
printPointList(convexHull)